# Basic Multiprocessing with Process
# Use case: Manual process management (custom process orchestration, low-level control)

In [ ]:
from multiprocessing import Process
import time
import os

def worker(task_id):
    print(f"[Process {task_id}] PID={os.getpid()} starting...")
    time.sleep(2)
    print(f"[Process {task_id}] finished.")

if __name__ == "__main__":
    processes = []

    for i in range(3):
        p = Process(target=worker, args=(i,))
        processes.append(p)
        p.start()

    for p in processes:
        p.join()

    print("✅ All processes complete.")


# Subclassing Process (OOP-style)
# Use case: Encapsulated logic per process (e.g. ML model per process)

In [ ]:
from multiprocessing import Process
import time

class MyProcess(Process):
    def __init__(self, name):
        super().__init__()
        self.name = name

    def run(self):
        print(f"Process {self.name} started.")
        time.sleep(1)
        print(f"Process {self.name} done.")

if __name__ == "__main__":
    processes = [MyProcess(i) for i in range(3)]
    for p in processes: p.start()
    for p in processes: p.join()


# Subclassing Process (OOP-style)
# Use case: Encapsulated logic per process (e.g. ML model per process)

In [ ]:
from multiprocessing import Process
import time

class MyProcess(Process):
    def __init__(self, name):
        super().__init__()
        self.name = name

    def run(self):
        print(f"Process {self.name} started.")
        time.sleep(1)
        print(f"Process {self.name} done.")

if __name__ == "__main__":
    processes = [MyProcess(i) for i in range(3)]
    for p in processes: p.start()
    for p in processes: p.join()


# Using multiprocessing.Pool (Parallel Map)
# Use case: Parallelizing CPU-heavy data computations
# Advantage: Automatic process pool management, simple API



In [ ]:
from multiprocessing import Pool
import time, os

def square(n):
    print(f"Process {os.getpid()} → computing {n}²")
    time.sleep(1)
    return n * n

if __name__ == "__main__":
    with Pool(processes=4) as pool:
        results = pool.map(square, [1, 2, 3, 4, 5])
    print("Results:", results)


# Using Pool.apply() and Pool.apply_async()
# Use case: Async tasks, parallel computations with other ongoing operations

In [ ]:
from multiprocessing import Pool

# Synchronous (blocking)
def cube(n):
    return n ** 3

if __name__ == "__main__":
    with Pool(3) as pool:
        print("Result:", pool.apply(cube, (5,)))  # Waits for completion


# Asynchronous (non-blocking)
from multiprocessing import Pool

def cube(n):
    return n ** 3

if __name__ == "__main__":
    with Pool(3) as pool:
        result = pool.apply_async(cube, (5,))
        print("Doing other work...")
        print("Result:", result.get())  # Fetch result later


# Using concurrent.futures.ProcessPoolExecutor (Recommended Modern API)
# ✅ Thread-safe, Pythonic, production-friendly


In [ ]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import time, math

def heavy_computation(x):
    time.sleep(1)
    return math.factorial(x)

if __name__ == "__main__":
    with ProcessPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(heavy_computation, n) for n in [5, 6, 7, 8]]
        for future in as_completed(futures):
            print("Result:", future.result())


# Using multiprocessing.Queue (Inter-Process Communication)
#  ✅ Safely exchange data between processes
# Use case: Data pipelines, message passing, task queues

In [ ]:
from multiprocessing import Process, Queue
import time

def producer(q):
    for i in range(5):
        print("Producing:", i)
        q.put(i)
        time.sleep(1)
    q.put(None)  # Signal end

def consumer(q):
    while True:
        item = q.get()
        if item is None:
            break
        print("Consumed:", item)
        time.sleep(2)

if __name__ == "__main__":
    q = Queue()
    Process(target=producer, args=(q,)).start()
    Process(target=consumer, args=(q,)).start()


# Using multiprocessing.Pipe (Two-way Communication)
# from multiprocessing import Process, Pipe

In [ ]:
from multiprocessing import Process, Pipe

def sender(conn):
    conn.send("Hello from child")
    conn.close()

def receiver(conn):
    msg = conn.recv()
    print("Received:", msg)

if __name__ == "__main__":
    parent_conn, child_conn = Pipe()
    Process(target=sender, args=(child_conn,)).start()
    Process(target=receiver, args=(parent_conn,)).start()


# Using Shared Memory (Value / Array)
# ✅ Share small data between processes (without serialization)

In [ ]:
from multiprocessing import Process, Value, Array

def update_data(num, arr):
    num.value += 1
    for i in range(len(arr)):
        arr[i] *= -1

if __name__ == "__main__":
    num = Value('i', 10)
    arr = Array('i', [1, 2, 3])

    p = Process(target=update_data, args=(num, arr))
    p.start()
    p.join()

    print("Number:", num.value)
    print("Array:", list(arr))


# Using multiprocessing.Manager (Share Complex Objects)

In [ ]:
from multiprocessing import Process, Manager

def worker(shared_dict, key, value):
    shared_dict[key] = value

if __name__ == "__main__":
    with Manager() as manager:
        shared_dict = manager.dict()
        processes = [Process(target=worker, args=(shared_dict, i, i*i)) for i in range(5)]

        for p in processes: p.start()
        for p in processes: p.join()

        print("Shared dict:", dict(shared_dict))


In [ ]:
# Using multiprocessing.Lock (Synchronization)
from multiprocessing import Process, Lock

def safe_print(lock, msg):
    with lock:
        print(msg)

if __name__ == "__main__":
    lock = Lock()
    for i in range(5):
        Process(target=safe_print, args=(lock, f"Message {i}")).start()

